In [ ]:
# Importing common libraries
import pandas as pd
import numpy as np
import requests
import time
from pathlib import Path
from tqdm.auto import tqdm
from unidecode import unidecode

# Importing geo libraries
import urbanpy as up
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# Reading the data
df = pd.read_excel(
    "inputs/encuesta_limpieza_DIRECCIONES LIMPIAS CON LINK EN MAPS.xlsx", index_col=0
)

In [ ]:
# Check the number of rows and columns
df.shape

In [ ]:
# Columnas relacionadas a la ubicacion geografica
df.columns[4:12]

In [ ]:
# Check clean addresses values
df["DIRECCION LIMPIA"].value_counts()

In [ ]:
# Check clean addresses number of null values
df["DIRECCION LIMPIA"].isna().sum()

In [ ]:
# Check types of links
df["LINK"].value_counts()

In [ ]:
# Check number of null values in links
df["LINK"].isna().sum()

In [ ]:
df["DIRECCION LIMPIA"][df["LINK"] == "NO SE ENCUENTRA"].value_counts()

In [ ]:
df["DIRECCION LIMPIA"][df["LINK"] == "YA GEORREFERENCIADA"].value_counts()

In [ ]:
df[df["DIRECCION LIMPIA"] == "BÚSQUEDA MANUAL"]["LINK"].isna().sum()

In [ ]:
# Extrae las coordenas de los links de busqueda de OSM
search_df = df[
    df["LINK"].fillna("NAN").str.startswith("https://www.openstreetmap.org/search")
].copy()
search_df[["lat", "lon"]] = search_df.apply(
    lambda x: (
        x["LINK"]
        .split("?")[1]
        .split("#")[0]
        .split("&")[1]
        .split("query=")[1]
        .split("%2C")
    ),
    axis=1,
    result_type="expand",
)

In [ ]:
search_df.shape

In [ ]:
# Crea un GeoDataFrame con las coordenadas extraidas
search_gdf = gpd.GeoDataFrame(
    data=search_df,
    geometry=gpd.points_from_xy(search_df["lon"], search_df["lat"], crs=4326),
)

In [ ]:
# Elimina las filas que ya tienen coordenadas
not_search = df.drop(search_df.index)

In [ ]:
# Filtra las filas que tiene links de objetos de OSM
df_osm_objects = not_search[
    not_search["LINK"].fillna("NAN").str.startswith("https://")
].copy()

In [ ]:
df_osm_objects.shape

In [ ]:
# Extrae el tipo de objeto de OSM (node = N, way = W, relation = R)
df_osm_objects["OSM_OBJ_TYPE"] = df_osm_objects["LINK"].apply(
    lambda x: (x.split("/")[3][0]).upper()
)
# Extrae el ID del objeto de OSM
df_osm_objects["OSM_IDS"] = df_osm_objects["LINK"].apply(
    lambda x: (x.split("/")[4].split("#")[0])
)
# Une el tipo de objeto y el ID para hacer la consulta a Nominatim API
df_osm_objects["OSM_QUERY"] = df_osm_objects["OSM_OBJ_TYPE"] + df_osm_objects["OSM_IDS"];

In [ ]:
# Verifica el formato de la consulta
df_osm_objects["OSM_QUERY"].head()

Example OSM lookup query URL: https://nominatim.openstreetmap.org/lookup?osm_ids=R146656,W104393803,N240109189

In [ ]:
def osm_lookup(df, obj_type):
    """
    Queries the OSM Nominatim API for the coordinates of a list of OSM objects.
    The query is done in chunks of 50 ids.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame with the OSM ids to query.
    obj_type : str
        Type of OSM object to query. Can be 'N', 'W' or 'R'.

    Returns
    -------
    all_data_df : pandas.DataFrame
        DataFrame with the coordinates and address of the OSM objects.
    """

    N = 50  # Up to 50 ids can be queried at the same time.

    dff = df[df["OSM_OBJ_TYPE"] == obj_type].copy()
    query_chunks = [dff[i : i + N] for i in range(0, len(dff), N)]

    query_responses = []
    for chunk in query_chunks:
        list_ids = ",".join(chunk["OSM_QUERY"].tolist())
        url = f"https://nominatim.openstreetmap.org/lookup?osm_ids={list_ids}&format=json&extratags=1"
        res = requests.get(url)
        try:
            data = res.json()
            query_responses.append(data)
        except:
            pass
        time.sleep(1)  # maximum of 1 request per second.

    all_data = []
    for q in query_responses:
        for l in q:
            all_data.append([l["lat"], l["lon"], l["display_name"]])

    all_data_df = pd.DataFrame(
        all_data, columns=["lat", "lon", "osm_address"], index=dff.index
    )

    return all_data_df

In [ ]:
# Use nominatim API to get the coordinates of the OSM objects

fp = Path("outputs/clean_address_osm_ids_correct.geojson")

if fp.is_file():
    gdf = gpd.read_file(fp)
else:
    all_nodes_df = osm_lookup(df_osm_objects, "N")
    all_ways_df = osm_lookup(df_osm_objects, "W")
    all_rel_df = osm_lookup(df_osm_objects, "R")

    df_w_address = pd.concat([all_nodes_df, all_ways_df, all_rel_df])
    df_complete = df_osm_objects.merge(df_w_address, left_index=True, right_index=True)

    gdf = gpd.GeoDataFrame(
        data=df_complete,
        geometry=gpd.points_from_xy(df_complete["lon"], df_complete["lat"], crs=4326),
    )

    gdf.to_file(fp, driver="GeoJSON")

In [ ]:
gdf.head()

In [ ]:
(df["LINK"] == "YA GEORREFERENCIADA").sum()

In [ ]:
df_osm_objects.shape, gdf.shape, df.shape

In [ ]:
n_new_georef = search_gdf.shape[0] + gdf.shape[0]
n_new_georef

In [ ]:
n_prev_georef = (df["LINK"] == "YA GEORREFERENCIADA").sum()
n_prev_georef, n_new_georef + n_prev_georef

In [ ]:
(n_new_georef + n_prev_georef) / df.shape[0]

In [ ]:
df.shape[0] - (n_new_georef + n_prev_georef)

In [ ]:
prev_geo_df = pd.read_excel("outputs/datos_georef.xlsx", index_col=0)

In [ ]:
prev_geo_gdf = gpd.GeoDataFrame(
    data=prev_geo_df,
    geometry=gpd.points_from_xy(prev_geo_df["lon"], prev_geo_df["lat"], crs=4326),
)

In [ ]:
prev_geo_gdf.shape, n_prev_georef

In [ ]:
# Check two extra
prev_geo_gdf.index[
    ~prev_geo_gdf.index.isin(df[df["LINK"] == "YA GEORREFERENCIADA"].index)
]

In [ ]:
df.loc[1615]["LINK"]

In [ ]:
df.loc[1699]["DIRECCION LIMPIA"]

In [ ]:
ica = up.download.nominatim_osm("Ica, Peru")
ica.crs = 4326

In [ ]:
ax = ica.plot(facecolor="none")
prev_geo_gdf.plot(color="red", ax=ax)
search_gdf.plot(color="blue", ax=ax)
gdf.plot(color="green", ax=ax)

In [ ]:
## Fix point at the south (near La Serena, Chile)
bad_point = search_df.iloc[search_df["lat"].astype(float).argmin()]

search_gdf.loc[bad_point.name, "lat"] = -13.83448
search_gdf.loc[bad_point.name, "lon"] = -76.24896
search_gdf.loc[bad_point.name, "geometry"] = Point(-76.24896, -13.83448)

In [ ]:
search_gdf.shape

In [ ]:
bad_points_in_lima = search_gdf[~search_gdf.intersects(ica.loc[0, "geometry"])]

In [ ]:
bad_points_in_lima.index

In [ ]:
search_gdf.loc[745, "lat"] = -14.064057
search_gdf.loc[745, "lon"] = -75.740620
search_gdf.loc[745, "geometry"] = Point(-75.740620, -14.064057)

search_gdf.loc[1354, "lat"] = -14.084670
search_gdf.loc[1354, "lon"] = -75.723141
search_gdf.loc[1354, "geometry"] = Point(-75.723141, -14.084670)

In [ ]:
gdf_ALL = pd.concat([prev_geo_gdf, search_gdf, gdf])

In [ ]:
gdf_ALL = gdf_ALL[~gdf_ALL.index.duplicated(keep="last")]

In [ ]:
gdf_ALL.shape

In [ ]:
1085 / 1813

In [ ]:
611 + 207 + 144

In [ ]:
1085 - 962

In [ ]:
gdf_ALL.to_file("outputs/datos_georef_completo.geojson", driver="GeoJSON")

# END

In [ ]:
ax = ica.plot(facecolor="none")
gdf_ALL.plot(ax=ax)

In [ ]:
ica_dist = up.download.nominatim_osm("Ica, Ica, Peru", 1)
pisco_dist = up.download.nominatim_osm("Pisco, Ica, Peru", 1)
chincha_dist = up.download.nominatim_osm("Chincha, Ica, Peru")

In [ ]:
gdf_ALL["Provincia donde vive"].value_counts()

In [ ]:
gdf_ica = gdf_ALL[gdf_ALL["Provincia donde vive"] == "Ica"]
gdf_pisco = gdf_ALL[gdf_ALL["Provincia donde vive"] == "Pisco"]
gdf_chincha = gdf_ALL[gdf_ALL["Provincia donde vive"] == "Chincha"]

In [ ]:
import contextily as cx

In [ ]:
ax = ica.plot(facecolor="none", edgecolor="red", figsize=(10, 10))
ica_dist.plot(facecolor="none", ax=ax)
chincha_dist.plot(facecolor="none", ax=ax)

gdf_ALL.plot(ax=ax)

pisco_dist.plot(facecolor="none", ax=ax)
ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs=4326)

In [ ]:
ax = ica.plot(facecolor="none", edgecolor="red", figsize=(10, 10))
ica_dist.plot(facecolor="none", ax=ax)
chincha_dist.plot(facecolor="none", ax=ax)

gdf_ica.plot(color="red", ax=ax)
gdf_chincha.plot(color="blue", ax=ax)
gdf_pisco.plot(color="green", ax=ax)

pisco_dist.plot(facecolor="none", ax=ax)
ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs=4326)

In [ ]:
districts = gpd.read_file(
    "https://storage.googleapis.com/up_public_geodata/admin_bounds/peru/districts.zip",
    mask=ica,
)

In [ ]:
ica_districts = districts[districts["DEPARTAMEN"] == "ICA"]

In [ ]:
ax = ica.plot(facecolor="none")
ica_districts.plot(ax=ax)

In [ ]:
ax = up.geom.merge_shape_hex(ica_districts, gdf_ALL, agg={"lat": "size"}).plot(
    "lat", legend=True
)
gdf_ALL.plot(color="r", ax=ax)

In [ ]:
(
    gdf_ALL.intersects(ica_dist.iloc[0].geometry).sum(),
    gdf_ALL.intersects(chincha_dist.iloc[0].geometry).sum(),
    gdf_ALL.intersects(pisco_dist.iloc[0].geometry).sum(),
)

In [ ]:
hexs = up.geom.gen_hexagons(6, ica)
hexs.plot()

In [ ]:
hexs_pip = up.geom.merge_shape_hex(hexs, gdf_ALL, agg={"lat": "size"})

hexs_pip.plot("lat", legend=True)

In [ ]:
# INSEGURIDAD ALIMENTARIA
rename_cols = {
    "En los últimos 12 meses ¿Usted u otra persona en su hogar se haya preocupado por no tener suficientes alimentos para comer por falta de dinero u otros recursos?": "Preocupación por no tener alimentos (Leve)",
    "Pensando aún en los últimos 12 meses, ¿hubo alguna vez en que usted u otra persona en su hogar no haya podido comer alimentos saludables y nutritivos  por falta de dinero u otros recursos?": "No pudo comer alimentos saludables y nutritivos (Moderada)",
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya comido poca variedad de  alimentos por falta de dinero u otros recursos?": "Comió poca variedad de alimentos (Moderada)",
    "En los últimos 12 meses Pensando aún en los últimos 12 meses, ¿hubo alguna vez en que usted u otra persona en su hogar haya comido menos de lo que pensaba que debía comer por falta de dinero u otros recursos?": "Comió menos de lo que pensaba que debía (Moderada)",
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya tenido que dejar de desayunar, almorzar o cenar porque no había suficiente dinero u otros recursos para obtener alimentos?": "Se saltó alguna comida por falta de dinero para comprar alimentos (Moderada)",
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya sentido  hambre pero no comió porque no había suficiente dinero u otros recursos para obtener alimentos?": "Tuvo hambre pero no comió por falta de alientos (Moderada)",
    "En los últimos 12 meses ¿Hubo alguna vez en que su hogar se haya quedado sin alimentos por falta de dinero u otros recursos?": "Se quedó sin alimentos (Grave)",
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya dejado de comer todo un día por falta de dinero u otros recursos?": "Dejo de comer todo un día (Grave)",
}

In [ ]:
cols = list(rename_cols.values())
cols.reverse()

In [ ]:
gdf_ALL = gdf_ALL.rename(columns=rename_cols)

In [ ]:
def get_insec_alim(row):
    for c in cols:
        if row[c] == "Si":
            return c.split("(")[1][:-1]
    return np.nan

In [ ]:
gdf_ALL["inseg_alim_nivel"] = gdf_ALL.apply(get_insec_alim, axis=1)

In [ ]:
gdf_ALL["inseg_alim_nivel"].value_counts()

In [ ]:
gdf_ALL["inseg_alim_nivel"].isna().sum()

In [ ]:
# Zona Chincha
hexs_chincha = hexs_pip.cx[-78:-76, -13.6:-13]
ax = hexs_chincha.plot("lat", legend=True)
gdf_ALL.clip(hexs_chincha).plot(alpha=0.5, ax=ax)

In [ ]:
gdf_zona_chincha = gdf_ALL.clip(hexs_chincha)

In [ ]:
# Zona Pisco
hexs_pisco = hexs_pip.cx[-78:-76, -14.0:-13.6]
ax = hexs_pisco.plot("lat", legend=True)
gdf_ALL.clip(hexs_pisco).plot(alpha=0.5, ax=ax)

In [ ]:
gdf_zona_pisco = gdf_ALL.clip(hexs_pisco)

In [ ]:
# Zona ICA
hexs_ica = hexs_pip.cx[-76:-75.5, -14.5:-14]
ax = hexs_ica.plot("lat", legend=True)
gdf_ALL.clip(hexs_ica).plot(alpha=0.5, ax=ax)

In [ ]:
gdf_zona_ica = gdf_ALL.clip(hexs_ica)

--- MENCIONADOS EN INFORME ANTERIOR ---
1. Mapas de inseguridad alimentaria para cada nivel 
2. Mapa de hogares que recibieron QW categorizado por si fue significativo o no
3. Mapa con personas que estan dispuestas a organizarse
4. Mapa con hogares por tipo de situación migratoria
5. MApa con hogares por ingreso  

---- PUEDO AGREGAR ---  
1. Mapa hogares sobre densidad poblacional 
1. Mapa de hogares sobre caminabilidad a mercados (CENSO de mcdos 2016)
1. Mapa de hogares sobre caminabilidad a centros de salud (Datos Abiertos de OpenStreetMap)

## Mapas de inseguridad alimentaria para cada nivel 


In [ ]:
from shapely.geometry import Polygon
import contextily as cx
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt

In [ ]:
minx, miny, maxx, maxy = -76.2, -13.45, -76.1, -13.38


def bbox_to_gdf(minx, miny, maxx, maxy):
    geometry = [Polygon([[minx, miny], [minx, maxy], [maxx, maxy], [maxx, miny]])]
    return gpd.GeoSeries(geometry, crs=4326)

In [ ]:
ica_districts["PROVINCIA"].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 10))

ica_districts.plot("PROVINCIA", alpha=0.5, legend=True, ax=ax)

gdf_ALL.plot(color="orange", ax=ax)

bbox_to_gdf(*hexs_chincha.dropna().total_bounds).plot(
    facecolor="none", edgecolor="r", ax=ax
)
bbox_to_gdf(*hexs_pisco.dropna().total_bounds).plot(
    facecolor="none", edgecolor="b", ax=ax
)
bbox_to_gdf(*hexs_ica.dropna().total_bounds).plot(
    facecolor="none", edgecolor="g", ax=ax
)

ax.set_axis_off()
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs=4326)

In [ ]:
# ICA
gdf_ica = gdf_ALL.clip(
    ica_districts[ica_districts["PROVINCIA"] == "ICA"].unary_union
).cx[-75.77531:-75.67972, -14.15:-13.970591]
# CHINCHA
gdf_chincha = gdf_ALL.clip(
    ica_districts[
        (ica_districts["PROVINCIA"] == "CHINCHA")
        & (ica_districts["DISTRITO"] != "SAN PEDRO DE HUACARPANA")
    ].unary_union
).cx[-76.1818:-76.10, -13.45:-13.39094]
# PISCO
gdf_pisco = gdf_ALL.clip(
    ica_districts[ica_districts["PROVINCIA"] == "PISCO"].unary_union
).cx[-76.25:-76.1, -13.78:-13.65]
# .cx[-76.2:-76.1, -13.78:-13.65]
# gdf_pisco: [-76.29258972 -13.8375251  -76.0048     -13.6229    ]

print("gdf_ica:", gdf_ica.total_bounds)
print("gdf_chincha:", gdf_chincha.total_bounds)
print("gdf_pisco:", gdf_pisco.total_bounds)

# Plot every gdf
gdf_ica.plot("inseg_alim_nivel", legend=True)
gdf_chincha.plot("inseg_alim_nivel", legend=True)
gdf_pisco.plot("inseg_alim_nivel", legend=True)

In [ ]:
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.lines import Line2D

In [ ]:
points = gpd.GeoSeries(
    [Point(-75, -13.75), Point(-76, -13.75)], crs=4326
)  # Geographic WGS 84 - degrees
points = points.to_crs(32618)  # Projected WGS 84 - meters

In [ ]:
distance_meters = points[0].distance(points[1])

In [ ]:
distance_meters

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 10))

ica_districts[ica_districts["PROVINCIA"].isin(["CHINCHA", "PISCO", "ICA"])].plot(
    alpha=0.5, facecolor="none", label="Encuestas", legend=True, ax=ax
)

gdf_ica.plot(color="g", ax=ax)
gdf_chincha.plot(color="g", ax=ax)
gdf_pisco.plot(color="g", ax=ax)

bbox_to_gdf(*gdf_ica.total_bounds).plot(
    facecolor="none", edgecolor="r", linewidth=3, ax=ax
)
bbox_to_gdf(*gdf_chincha.total_bounds).plot(
    facecolor="none", edgecolor="orange", linewidth=3, ax=ax
)
bbox_to_gdf(*gdf_pisco.total_bounds).plot(
    facecolor="none", edgecolor="k", linewidth=3, ax=ax
)

minx, miny, maxx, maxy = -76.4, -14.25, -75.6, -13.25
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

ax.add_artist(ScaleBar(distance_meters, box_alpha=0.5, location="lower left"))

g_patch = Line2D([0], [0], marker="o", color="g", linewidth=0, label="Observaciones")

r_line = Line2D([0], [0], color="r", linewidth=3, label="Ica")
b_line = Line2D([0], [0], color="k", linewidth=3, label="Pisco")
o_line = Line2D([0], [0], color="orange", linewidth=3, label="Chincha")

plt.legend(handles=[o_line, b_line, r_line, g_patch], title="Zonas de análisis")

ax.set_axis_off()
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs=4326)

plt.savefig("outputs/maps/zonas_de_analisis.png", dpi=300, bbox_inches="tight")

In [ ]:
hexs_ica9 = up.geom.gen_hexagons(9, ica)

In [ ]:
hexs_ica9.shape

In [ ]:
gdf_ALL = gdf_ALL.assign(**pd.get_dummies(gdf_ALL["inseg_alim_nivel"]).to_dict())

In [ ]:
gdf_ALL["inseg_alim_nivel"].value_counts()

In [ ]:
gdf_ica["inseg_alim_nivel"] = gdf_ALL.loc[gdf_ica.index, "inseg_alim_nivel"]
gdf_chincha["inseg_alim_nivel"] = gdf_ALL.loc[gdf_ica.index, "inseg_alim_nivel"]
gdf_pisco["inseg_alim_nivel"] = gdf_ALL.loc[gdf_ica.index, "inseg_alim_nivel"]

In [ ]:
hexs_ica_inseg = up.geom.merge_shape_hex(
    hexs_ica9, gdf_ALL, agg={"Grave": "sum", "Moderada": "sum", "Leve": "sum"}
)

In [ ]:
hexs_ica_inseg.isna().sum() / hexs_ica_inseg.shape[0]

In [ ]:
# Create a function to plot a heatmap
def plot_heatmap(df, column, title, cmap="Reds", vmin=0, vmax=1, figsize=(20, 10)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.set_title(title)
    df.plot(column=column, cmap=cmap, vmin=vmin, vmax=vmax, ax=ax, legend=True)
    ax.set_axis_off()
    cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs=4326)
    plt.savefig(f"outputs/maps/{column}.png", dpi=300, bbox_inches="tight")

In [ ]:
# Hex Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
inseg_niveles = ["Grave", "Moderada", "Leve"]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Nivel de Inseguridad Alimentaria ({ciudades[i]})")
    for j, ax in enumerate(axes):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Nivel: {inseg_niveles[j]}")

        if j == 2:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            hexs_ica_inseg.cx[minx:maxx, miny:maxy].plot(
                inseg_niveles[j],
                edgecolor="none",
                alpha=0.75,
                cmap="YlOrRd",
                ax=ax,
                vmin=0,
                vmax=20,
                legend=True,
                cax=cax,
            )
        else:
            hexs_ica_inseg.cx[minx:maxx, miny:maxy].plot(
                inseg_niveles[j],
                edgecolor="none",
                alpha=0.75,
                cmap="YlOrRd",
                ax=ax,
                vmin=0,
                vmax=20,
            )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        if j == 0:
            ax.add_artist(
                ScaleBar(distance_meters, box_alpha=0.5, location="lower left")
            )

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.Positron,
            crs="EPSG:4326",
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps/{ciudades[i]}_inseg_alim.png", dpi=300, bbox_inches="tight"
    )

In [ ]:
import geoplot as gplt

In [ ]:
import seaborn as sns

In [ ]:
# Heat Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
inseg_niveles = ["Grave", "Moderada", "Leve"]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Nivel de Inseguridad Alimentaria ({ciudades[i]})")
    for j, ax in enumerate(axes):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Nivel: {inseg_niveles[j]}")

        gplt.kdeplot(
            gdf.query(f"inseg_alim_nivel == '{inseg_niveles[j]}'"),
            cmap="turbo",
            ax=ax,
            alpha=0.5,
            n_levels=100,
            fill=True,
            thresh=0.5,
        )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        if j == 0:
            ax.add_artist(
                ScaleBar(distance_meters, box_alpha=0.5, location="lower left")
            )

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
            crs="EPSG:4326",
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps/density/{ciudades[i]}_inseg_alim.png",
        dpi=300,
        bbox_inches="tight",
    )

In [ ]:
# Point Map
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle(f"Nivel de Inseguridad Alimentaria")

gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
inseg_niveles = ["Grave", "Moderada", "Leve"]

for j, ax in enumerate(axes):
    ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
    minx, miny, maxx, maxy = gdfs[j].total_bounds

    ax.set_title(f"Ciudad: {ciudades[j]}")

    if j == 2:
        gdfs[j].plot(
            "inseg_alim_nivel",
            alpha=1,
            cmap="Reds_r",
            ax=ax,
            categories=["Grave", "Moderada", "Leve"],
            legend=True,
            legend_kwds={"loc": "lower left", "bbox_to_anchor": (1, 0.5)},
        )
    else:
        gdfs[j].plot(
            "inseg_alim_nivel",
            alpha=1,
            cmap="Reds_r",
            ax=ax,
            categories=["Grave", "Moderada", "Leve"],
        )

    margins = 0.001
    ax.set_xlim(minx - margins, maxx + margins)
    ax.set_ylim(miny - margins, maxy + margins)

    if j == 0:
        ax.add_artist(ScaleBar(distance_meters, box_alpha=0.5, location="lower left"))

    ax.set_axis_off()
    cx.add_basemap(
        ax=ax,
        source="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
        crs="EPSG:4326",
        attribution=False,
    )

plt.tight_layout()
plt.savefig(f"outputs/maps/inseg_alim_puntos.png", dpi=300, bbox_inches="tight")

## Mapa de hogares que recibieron QW categorizado por si fue significativo o no

In [ ]:
gdf_ALL[
    [
        "Si tiene hijos en edad escolar, ¿Alguno de sus hijos recibe Qali Warma?",
        "¿Los alimentos recibidos por Qali Warma rma son un aporte importante a la alimentacion familiar?",
        "¿Que modalidad de Qali Warma recibe su hijo?",
    ]
]

In [ ]:
gdf_ALL[
    "Si tiene hijos en edad escolar, ¿Alguno de sus hijos recibe Qali Warma?"
].value_counts()

In [ ]:
gdf_ALL[
    "¿Los alimentos recibidos por Qali Warma rma son un aporte importante a la alimentacion familiar?"
].value_counts()

In [ ]:
gdf_ALL[
    (
        gdf_ALL[
            "Si tiene hijos en edad escolar, ¿Alguno de sus hijos recibe Qali Warma?"
        ]
        == "SÍ"
    )
    & (
        gdf_ALL[
            "¿Los alimentos recibidos por Qali Warma rma son un aporte importante a la alimentacion familiar?"
        ]
        == "SÍ"
    )
]

In [ ]:
gdf_ALL["tiene_qw"] = (
    gdf_ALL["Si tiene hijos en edad escolar, ¿Alguno de sus hijos recibe Qali Warma?"]
    == "SÍ"
).astype(int)
gdf_ALL["qw_importante"] = (
    gdf_ALL[
        "¿Los alimentos recibidos por Qali Warma rma son un aporte importante a la alimentacion familiar?"
    ]
    == "SÍ"
).astype(int)

In [ ]:
hexs_ica_inseg = up.geom.merge_shape_hex(
    hexs_ica9, gdf_ALL, agg={"tiene_qw": "sum", "qw_importante": "sum"}
)

In [ ]:
# Hex Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
vars = ["tiene_qw", "qw_importante"]
labels = [
    "Familias que reciben Qaliwarma",
    "Los alimentos de Qaliwarma son importantes\npara la alimentación familiar",
]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(f"{ciudades[i]}")

    for j, ax in enumerate(axes):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)

        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"{labels[j]}")

        if j == 1:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            hexs_ica_inseg.cx[minx:maxx, miny:maxy].plot(
                vars[j],
                edgecolor="none",
                alpha=0.75,
                cmap="YlOrRd",
                ax=ax,
                vmin=0,
                vmax=10,
                legend=True,
                cax=cax,
            )
        else:
            hexs_ica_inseg.cx[minx:maxx, miny:maxy].plot(
                vars[j],
                edgecolor="none",
                alpha=0.75,
                cmap="YlOrRd",
                ax=ax,
                vmin=0,
                vmax=10,
            )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        if j == 0:
            ax.add_artist(
                ScaleBar(distance_meters, box_alpha=0.5, location="lower left")
            )

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.Positron,
            crs="EPSG:4326",
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps/{ciudades[i]}_qaliwarma.png", dpi=300, bbox_inches="tight"
    )

In [ ]:
# Heat Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
vars = ["tiene_qw", "qw_importante"]
labels = [
    "Familias que reciben Qaliwarma",
    "Los alimentos de Qaliwarma son importantes\npara la alimentación familiar",
]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(f"{ciudades[i]}")

    for j, ax in enumerate(axes):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)

        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"{labels[j]}")

        gplt.kdeplot(
            gdf.query(f"{vars[j]} == 1"),
            cmap="Reds",
            ax=ax,
            alpha=0.5,
            fill=True,
            n_levels=100,
            thresh=0.5,
        )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        if j == 0:
            ax.add_artist(
                ScaleBar(distance_meters, box_alpha=0.5, location="lower left")
            )

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.Positron,
            crs="EPSG:4326",
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps/density/{ciudades[i]}_qaliwarma.png",
        dpi=300,
        bbox_inches="tight",
    )

## Mapa con personas que estan dispuestas a organizarse 

In [ ]:
gdf_ALL["En caso no, ¿Estaria dispuesto en participar en una?"].value_counts()

In [ ]:
gdf_ALL["dispuesto_organizarse"] = (
    gdf_ALL["En caso no, ¿Estaria dispuesto en participar en una?"] == "SÍ"
).astype(int)

In [ ]:
hexs_ica_inseg = up.geom.merge_shape_hex(
    hexs_ica9, gdf_ALL, agg={"dispuesto_organizarse": "sum"}
)

In [ ]:
# Hex Map
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Mapa con personas que estan dispuestas a organizarse")

gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]

for j, ax in enumerate(axes):
    ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
    minx, miny, maxx, maxy = gdfs[j].total_bounds

    ax.set_title(f"Ciudad: {ciudades[j]}")

    if j == 2:
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        hexs_ica_inseg.cx[minx:maxx, miny:maxy].plot(
            "dispuesto_organizarse",
            alpha=0.75,
            cmap="YlOrRd",
            ax=ax,
            vmin=0,
            vmax=30,
            legend=True,
            cax=cax,
            edgecolor="none",
        )
    else:
        hexs_ica_inseg.cx[minx:maxx, miny:maxy].plot(
            "dispuesto_organizarse",
            alpha=0.75,
            cmap="YlOrRd",
            ax=ax,
            vmin=0,
            vmax=30,
            edgecolor="none",
        )

    margins = 0.001
    ax.set_xlim(minx - margins, maxx + margins)
    ax.set_ylim(miny - margins, maxy + margins)

    if j == 0:
        ax.add_artist(ScaleBar(distance_meters, box_alpha=0.5, location="lower left"))

    ax.set_axis_off()
    cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.tight_layout()
plt.savefig(f"outputs/maps/dispuestas_organizarse.png", dpi=300, bbox_inches="tight")

In [ ]:
# Heat Map
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Mapa con personas que estan dispuestas a organizarse")

gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]

for j, ax in enumerate(axes):
    ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
    minx, miny, maxx, maxy = gdfs[j].total_bounds

    ax.set_title(f"Ciudad: {ciudades[j]}")

    gplt.kdeplot(
        gdfs[j].query(f"dispuesto_organizarse == 1"),
        cmap="Reds",
        ax=ax,
        alpha=0.5,
        fill=True,
        thresh=0.05,
    )

    margins = 0.001
    ax.set_xlim(minx - margins, maxx + margins)
    ax.set_ylim(miny - margins, maxy + margins)

    ax.set_axis_off()
    cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.tight_layout()
plt.savefig(
    f"outputs/maps/density/dispuestas_organizarse.png", dpi=300, bbox_inches="tight"
)

## Mapa con hogares por tipo de situación migratoria

In [ ]:
gdf_ALL["¿Cuál es SU actual ESTATUS MIGRATORIO?"].value_counts()

In [ ]:
gdf_ALL["status_migratorio"] = (
    gdf_ALL["¿Cuál es SU actual ESTATUS MIGRATORIO?"]
    .replace({"Otro": np.nan})
    .fillna(gdf_ALL["Especifica otro estatus migratorio"])
)

In [ ]:
gdf_ALL["status_migratorio"].value_counts()

In [ ]:
gdf_ALL["status_migratorio"].unique()

In [ ]:
gdf_ALL = gdf_ALL.assign(**pd.get_dummies(gdf_ALL["status_migratorio"]).to_dict())

In [ ]:
# Check integrity of value counts
gdf_ALL[
    [
        "Carnet de Extranjeria",
        "CPP",
        "Mantengo mi documento de identidad venezolano",
        "No especificado",
        "Carnet de Solicitud de Refugio",
        "No posee documentos",
    ]
].sum()

In [ ]:
hexs_ica_inseg = up.geom.merge_shape_hex(
    hexs_ica9,
    gdf_ALL,
    agg={
        "Carnet de Extranjeria": "sum",
        "CPP": "sum",
        "Mantengo mi documento de identidad venezolano": "sum",
        "No especificado": "sum",
        "Carnet de Solicitud de Refugio": "sum",
        "No posee documentos": "sum",
    },
)

In [ ]:
import geoplot as gplt

In [ ]:
# # This is a temporary fix for geoplot
# !pip install cartopy==0.17.0
# !pip install geoplot

In [ ]:
# Hex Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
status_migratorio_options = [
    "Mantengo mi documento de identidad venezolano",
    "Carnet de Extranjeria",
    "CPP",
]
#    'Carnet de Solicitud de Refugio', 'Documento de identidad peruano',
#    'No posee documentos', 'No especificado',]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Estatus migratorio ({ciudades[i]})")
    for j, ax in enumerate(axes):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Estatus: {status_migratorio_options[j]}")

        if j == 2:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            hexs_ica_inseg.cx[minx:maxx, miny:maxy].plot(
                status_migratorio_options[j],
                edgecolor="none",
                alpha=1,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
                legend=True,
                cax=cax,
            )
        else:
            hexs_ica_inseg.cx[minx:maxx, miny:maxy].plot(
                status_migratorio_options[j],
                edgecolor="none",
                alpha=1,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
            )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.Positron,
            crs="EPSG:4326",
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps/{ciudades[i]}_status_migratorio.png",
        dpi=300,
        bbox_inches="tight",
    )

In [ ]:
# Heat Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
status_migratorio_options = [
    "Mantengo mi documento de identidad venezolano",
    "Carnet de Extranjeria",
    "CPP",
]
#    'Carnet de Solicitud de Refugio', 'Documento de identidad peruano',
#    'No posee documentos', 'No especificado',]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Estatus migratorio ({ciudades[i]})")
    for j, ax in enumerate(axes):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Estatus: {status_migratorio_options[j]}")

        gplt.kdeplot(
            gdf.query(f"status_migratorio == '{status_migratorio_options[j]}'"),
            cmap="Reds",
            ax=ax,
            alpha=0.5,
            fill=True,
            thresh=0.05,  # cbar=True, cbar_ax=cax,
        )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.Positron,
            crs="EPSG:4326",
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps/density/{ciudades[i]}_status_migratorio.png",
        dpi=300,
        bbox_inches="tight",
    )

## MApa con hogares por ingreso 

In [ ]:
gdf_ALL[
    "Sumando a TODOS los miembros de su hogar que perciben algun tipo de ingreso ¿Cuál es el INGRESO PROMEDIO MENSUAL  que reciben?"
].value_counts()

In [ ]:
gdf_ALL[
    "Sumando a TODOS los miembros de su hogar que perciben algun tipo de ingreso ¿Cuál es el INGRESO PROMEDIO MENSUAL  que reciben?"
].isna().sum()

In [ ]:
gdf_ALL[
    "Sumando a TODOS los miembros de su hogar que perciben algun tipo de ingreso ¿Cuál es el INGRESO PROMEDIO MENSUAL  que reciben?"
].unique()

In [ ]:
gdf_ALL["rango_ingreso"] = (
    gdf_ALL[
        "Sumando a TODOS los miembros de su hogar que perciben algun tipo de ingreso ¿Cuál es el INGRESO PROMEDIO MENSUAL  que reciben?"
    ]
    .replace({0: "0 a 300", None: np.nan})
    .fillna("No especificado")
)

In [ ]:
gdf_ALL["rango_ingreso"].unique()

In [ ]:
gdf_ALL = gdf_ALL.assign(**pd.get_dummies(gdf_ALL["rango_ingreso"]).to_dict())

In [ ]:
hexs_ica_inseg = up.geom.merge_shape_hex(
    hexs_ica9,
    gdf_ALL,
    agg={
        "0 a 300": "sum",
        "301 a 500": "sum",
        "501 a 1025 SALARIO BASICO": "sum",
        "1026 a 1500": "sum",
        "1501 A 2000": "sum",
        "2000 a más": "sum",
        "No especificado": "sum",
    },
)

In [ ]:
hexs_ica_inseg[ingreso_options].sum()

In [ ]:
# Hex Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
ingreso_options = [
    "0 a 300",
    "301 a 500",
    "501 a 1025 SALARIO BASICO",
    "1026 a 1500",
    "1501 A 2000",
    "2000 a más",
]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Ingreso promedio ({ciudades[i]})")
    for j, ax in enumerate(axes.flatten()):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Ingreso promedio: {ingreso_options[j]}")

        if j == 5:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            hexs_ica_inseg.cx[minx:maxx, miny:maxy].plot(
                ingreso_options[j],
                edgecolor="none",
                alpha=1,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
                legend=True,
                cax=cax,
            )
        else:
            hexs_ica_inseg.cx[minx:maxx, miny:maxy].plot(
                ingreso_options[j],
                edgecolor="none",
                alpha=1,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
            )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.Positron,
            crs="EPSG:4326",
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps/{ciudades[i]}_ingreso_promedio.png", dpi=300, bbox_inches="tight"
    )

In [ ]:
# Heat Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
ingreso_options = [
    "0 a 300",
    "301 a 500",
    "501 a 1025 SALARIO BASICO",
    "1026 a 1500",
    "1501 A 2000",
    "2000 a más",
]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Ingreso promedio ({ciudades[i]})")
    for j, ax in enumerate(axes.flatten()):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Ingreso promedio: {ingreso_options[j]}")

        gplt.kdeplot(
            gdf.query(f"rango_ingreso == '{ingreso_options[j]}'"),
            cmap="Reds",
            ax=ax,
            alpha=0.5,
            fill=True,
            thresh=0.05,  # cbar=True, cbar_ax=cax
        )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.Positron,
            crs="EPSG:4326",
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps/density/{ciudades[i]}_ingreso_promedio.png",
        dpi=300,
        bbox_inches="tight",
    )

## Mapa de densidad poblacional

In [ ]:
ica

In [ ]:
peru_pop = up.download.hdx_dataset(
    "4e74db39-87f1-4383-9255-eaf8ebceb0c9/resource/317f1c39-8417-4bde-a076-99bd37feefce/download/per_general_2020_csv.zip"
)
# https://data.humdata.org/dataset

In [ ]:
ica_pop = up.geom.filter_population(peru_pop, ica)

In [ ]:
ica_pop.head()

In [ ]:
hexs_ica9_pop = up.geom.merge_shape_hex(hexs_ica9, ica_pop, {"per_general_2020": "sum"})

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Mapa de densidad poblacional (hab./0.1km$^2$)")

gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]

for j, ax in enumerate(axes):
    ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
    minx, miny, maxx, maxy = gdfs[j].total_bounds

    ax.set_title(f"Ciudad: {ciudades[j]}")

    if j == 2:
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        hexs_ica9_pop.cx[minx:maxx, miny:maxy].query("per_general_2020 > 0").plot(
            "per_general_2020",
            alpha=0.6,
            ax=ax,
            cmap="Reds",
            vmin=0,
            vmax=2000,
            legend=True,
            cax=cax,
            edgecolor="none",
        )
    else:
        hexs_ica9_pop.cx[minx:maxx, miny:maxy].query("per_general_2020 > 0").plot(
            "per_general_2020",
            alpha=0.6,
            ax=ax,
            cmap="Reds",
            vmin=0,
            vmax=2000,
            edgecolor="none",
        )

    margins = 0.001
    ax.set_xlim(minx - margins, maxx + margins)
    ax.set_ylim(miny - margins, maxy + margins)

    ax.set_axis_off()
    cx.add_basemap(
        ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326", attribution=False
    )

plt.tight_layout()
plt.savefig(f"outputs/maps/densidad_poblacional.png", dpi=300, bbox_inches="tight")